## Ejemplo 1: Selección de Actividades

Seleccionar el máximo número de actividades que no se superpongan.

In [ ]:
def seleccion_actividades(actividades):
    """
    Selecciona máximo número de actividades no superpuestas.
    Estrategia voraz: ordenar por tiempo de fin
    Complejidad: O(n log n)
    """
    if not actividades:
        return []
    
    # CLAVE VORAZ: Ordenar por tiempo de finalización
    actividades_ordenadas = sorted(actividades, key=lambda x: x[1])
    
    seleccionadas = []
    tiempo_fin_actual = 0
    
    print("Procesando actividades en orden de finalización:\n")
    
    for inicio, fin, nombre in actividades_ordenadas:
        if inicio >= tiempo_fin_actual:
            seleccionadas.append((inicio, fin, nombre))
            tiempo_fin_actual = fin
            print(f"✓ {nombre}: [{inicio:2d}, {fin:2d}] - SELECCIONADA")
        else:
            print(f"✗ {nombre}: [{inicio:2d}, {fin:2d}] - Conflicto (termina en {tiempo_fin_actual})")
    
    return seleccionadas

# Prueba
actividades = [
    (1, 4, "A1"),
    (3, 5, "A2"),
    (0, 6, "A3"),
    (5, 7, "A4"),
    (3, 9, "A5"),
    (5, 9, "A6"),
    (6, 10, "A7"),
    (8, 11, "A8"),
    (8, 12, "A9"),
    (2, 14, "A10"),
    (12, 16, "A11")
]

print("PROBLEMA: Selección de Actividades")
print("="*60)

resultado = seleccion_actividades(actividades)

print(f"\n{'='*60}")
print(f"RESULTADO: {len(resultado)} actividades seleccionadas")
print(f"{'='*60}")
for inicio, fin, nombre in resultado:
    print(f"  {nombre}: [{inicio}, {fin}]")

## Ejemplo 2: Cambio de Monedas (Sistema Canónico)

El algoritmo voraz funciona para sistemas de monedas canónicos como USD, EUR.

In [ ]:
def cambio_voraz(monedas, cantidad):
    """
    Cambio usando estrategia voraz.
    NOTA: Solo óptimo para sistemas canónicos.
    Complejidad: O(n)
    """
    monedas_ordenadas = sorted(monedas, reverse=True)
    cambio = []
    cantidad_restante = cantidad
    
    print(f"Dando cambio de {cantidad}:\n")
    
    for moneda in monedas_ordenadas:
        count = cantidad_restante // moneda
        if count > 0:
            cambio.extend([moneda] * count)
            cantidad_restante -= moneda * count
            print(f"  {count} moneda(s) de {moneda}")
    
    if cantidad_restante > 0:
        return None
    
    return cambio

# Prueba con sistema canónico (USD)
monedas_usa = [25, 10, 5, 1]
cantidad = 63

print("Sistema Canónico (USD)")
print("="*60)
cambio = cambio_voraz(monedas_usa, cantidad)
print(f"\nTotal: {len(cambio)} monedas")
print(f"Cambio: {cambio}")

# Contraejemplo: sistema no canónico
print("\n" + "="*60)
print("CONTRAEJEMPLO: Sistema No Canónico")
print("="*60)
monedas_nc = [10, 7, 1]
cantidad_nc = 14
cambio_nc = cambio_voraz(monedas_nc, cantidad_nc)
print(f"\nCambio voraz: {cambio_nc} ({len(cambio_nc)} monedas)")
print(f"Solución óptima sería: [7, 7] (2 monedas)")
print("\n⚠️ Voraz NO garantiza óptimo en sistemas no canónicos")

## Ejemplo 3: Problema de la Mochila Fraccionaria

A diferencia de la mochila 0/1, aquí se pueden tomar fracciones de items.

In [ ]:
def mochila_fraccionaria(pesos, valores, capacidad):
    """
    Mochila fraccionaria usando algoritmo voraz.
    Estrategia: Ordenar por valor/peso y tomar items con mejor ratio.
    Complejidad: O(n log n)
    """
    n = len(pesos)
    
    # Crear lista de items con ratio valor/peso
    items = []
    for i in range(n):
        ratio = valores[i] / pesos[i]
        items.append((ratio, pesos[i], valores[i], i))
    
    # CLAVE VORAZ: Ordenar por ratio descendente
    items.sort(reverse=True)
    
    valor_total = 0
    peso_total = 0
    fracciones = [0] * n
    
    print(f"Capacidad: {capacidad}\n")
    print(f"{'Item':<6} {'Peso':<8} {'Valor':<8} {'Ratio':<10} {'Fracción':<10}")
    print("="*60)
    
    for ratio, peso, valor, i in items:
        if peso_total + peso <= capacidad:
            # Tomar item completo
            fracciones[i] = 1.0
            peso_total += peso
            valor_total += valor
            print(f"Item {i:<2} {peso:<8.1f} {valor:<8.1f} {ratio:<10.2f} 100%")
        else:
            # Tomar fracción
            espacio_restante = capacidad - peso_total
            fraccion = espacio_restante / peso
            fracciones[i] = fraccion
            peso_total += espacio_restante
            valor_total += valor * fraccion
            print(f"Item {i:<2} {peso:<8.1f} {valor:<8.1f} {ratio:<10.2f} {fraccion*100:.1f}%")
            break
    
    return valor_total, fracciones

# Prueba
pesos = [10, 20, 30]
valores = [60, 100, 120]
capacidad = 50

print("Problema de la Mochila Fraccionaria")
print("="*60)

valor_max, fracciones = mochila_fraccionaria(pesos, valores, capacidad)

print(f"\n{'='*60}")
print(f"Valor máximo: {valor_max:.2f}")
print(f"\nDetalle:")
for i, frac in enumerate(fracciones):
    if frac > 0:
        print(f"  Item {i}: {frac*100:.1f}% (peso={pesos[i]*frac:.1f}, valor={valores[i]*frac:.1f})")

## Ejemplo 4: Algoritmo de Huffman (Visualización Simplificada)

Construcción de códigos de compresión óptimos.

In [ ]:
import heapq
from collections import Counter

class NodoHuffman:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.izq = None
        self.der = None
    
    def __lt__(self, otro):
        return self.freq < otro.freq

def construir_codigos_huffman(texto):
    """
    Construye códigos de Huffman.
    Estrategia voraz: combinar dos nodos de menor frecuencia.
    """
    # Contar frecuencias
    frecuencias = Counter(texto)
    
    # Crear heap con nodos hoja
    heap = [NodoHuffman(char, freq) for char, freq in frecuencias.items()]
    heapq.heapify(heap)
    
    print(f"Texto: {texto}")
    print(f"\nFrecuencias:")
    for char, freq in sorted(frecuencias.items()):
        print(f"  '{char}': {freq}")
    
    # Construir árbol
    while len(heap) > 1:
        izq = heapq.heappop(heap)
        der = heapq.heappop(heap)
        
        padre = NodoHuffman(None, izq.freq + der.freq)
        padre.izq = izq
        padre.der = der
        
        heapq.heappush(heap, padre)
    
    # Generar códigos
    raiz = heap[0]
    codigos = {}
    
    def generar(nodo, codigo=""):
        if nodo.char is not None:
            codigos[nodo.char] = codigo or "0"
            return
        if nodo.izq:
            generar(nodo.izq, codigo + "0")
        if nodo.der:
            generar(nodo.der, codigo + "1")
    
    generar(raiz)
    return codigos

# Prueba
texto = "ABRACADABRA"

print("Códigos de Huffman")
print("="*60)

codigos = construir_codigos_huffman(texto)

print(f"\nCódigos generados:")
for char in sorted(codigos.keys()):
    print(f"  '{char}': {codigos[char]}")

# Calcular compresión
texto_codificado = ''.join(codigos[c] for c in texto)
bits_original = len(texto) * 8
bits_comprimido = len(texto_codificado)

print(f"\nCompresión:")
print(f"  Original: {bits_original} bits ({len(texto)} chars × 8 bits)")
print(f"  Comprimido: {bits_comprimido} bits")
print(f"  Ahorro: {(1 - bits_comprimido/bits_original)*100:.1f}%")

## 🎯 Ejercicio Práctico

Implementa un algoritmo voraz para minimizar la latencia máxima en scheduling.

In [ ]:
def minimizar_latencia(tareas):
    """
    Minimiza latencia máxima programando tareas.
    tareas: lista de (duración, deadline, nombre)
    Estrategia voraz: Ordenar por deadline (EDD - Earliest Deadline First)
    """
    # Ordenar por deadline
    tareas_ordenadas = sorted(tareas, key=lambda x: x[1])
    
    tiempo_actual = 0
    latencia_max = 0
    programacion = []
    
    for duracion, deadline, nombre in tareas_ordenadas:
        inicio = tiempo_actual
        fin = tiempo_actual + duracion
        latencia = max(0, fin - deadline)
        latencia_max = max(latencia_max, latencia)
        
        programacion.append((nombre, inicio, fin, deadline, latencia))
        tiempo_actual = fin
    
    return programacion, latencia_max

# Prueba
tareas = [
    (3, 6, "T1"),
    (2, 8, "T2"),
    (1, 9, "T3"),
    (4, 9, "T4"),
    (3, 14, "T5")
]

programacion, lat_max = minimizar_latencia(tareas)

print("Minimización de Latencia")
print("="*70)
print(f"{'Tarea':<8} {'Inicio':<8} {'Fin':<8} {'Deadline':<10} {'Latencia':<10}")
print("="*70)

for nombre, inicio, fin, deadline, latencia in programacion:
    lat_str = f"{latencia}" if latencia > 0 else "-"
    print(f"{nombre:<8} {inicio:<8} {fin:<8} {deadline:<10} {lat_str:<10}")

print(f"\nLatencia máxima: {lat_max}")

## 🎓 Conclusiones

- ✅ Algoritmos voraces son muy eficientes
- ⚠️ No siempre dan solución óptima
- 🔑 La clave es elegir la estrategia voraz correcta
- 📊 Generalmente O(n log n) por ordenamiento

**Siguiente:** [Backtracking](../05_Backtracking/README.md)